In [2]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os.path
import time
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import numpy as np
import random
from sklearn.metrics import mean_squared_error
from scipy.linalg import sqrtm
import qiskit.quantum_info as qi
from qiskit.providers.aer import extensions  # import aer snapshot instructions
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram
import qiskit.providers.aer.noise as noise
from qiskit.aqua.operators import StateFn
from qiskit.aqua.operators import Z
%matplotlib inline
from qiskit import Aer, IBMQ, execute
from qiskit.providers.aer import noise
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
import qiskit.providers.aer.extensions

In [3]:
times= np.linspace(0,1,1000)
#compare observables sigmaz
provider = IBMQ.load_account()
#provider.backends()

device = provider.get_backend('ibmq_manila')
properties = device.properties()
coupling_map = device.configuration().coupling_map

def eigenvalue_QT(omegax, omegay, omegaz, t, method):
    circ_N = QuantumCircuit(1, 1)
    circ_N.h(0)
    circ_N.barrier()
    circ_N.rx(t*omegax,0)
    circ_N.rz(2*t*omegaz,0)
    circ_N.ry(t*omegay,0)
    circ_N.snapshot_density_matrix('density_matrix')
    circ_N.measure([0],[0])
    #if noise==0:
    #Noise
    prob_1 = (1- np.exp(-gamma * t))  # 1-qubit gate
    # Depolarizing quantum errors
    error_1 = noise.depolarizing_error(prob_1, 1)

    # Add errors to noise model
    noise_model = noise.NoiseModel()
    noise_model.add_all_qubit_quantum_error(error_1, 'measure')
    #noise_model.add_all_qubit_quantum_error(error_2, ['cx'])
    # Get basis gates from noise model
    basis_gates = noise_model.basis_gates
    result_N = execute(circ_N, Aer.get_backend('qasm_simulator'),
             basis_gates=basis_gates,coupling_map=coupling_map,noise_model=noise_model,shots=2**16).result()
    if method==0:    
        rho_N = result_N.data()['snapshots']['density_matrix']['density_matrix'][0]['value']
        ev = np.trace(rho_N @ qt.sigmaz().full())
    elif method==1:
        psi_N = StateFn(result_N)
        ev = (~psi_N @ Z @ psi_N).eval()
    elif method==2:
        shots = result_N.results[0].shots
        counts = result_N.get_counts(circ_N)
        probs = ([(i, c/shots) for i, c in counts.items()])
        P = np.float64(np.array(probs)[:, 1])
        ev =  P[0] -P[1]
    return ev.real

IBMQAccountCredentialsNotFound: 'No IBM Quantum Experience credentials found.'